In [1]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()

# Get the current year and month
present_year = current_datetime.year
present_month = current_datetime.strftime("%B")
print(present_year, present_month)

2024 November


In [2]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate

# from pydantic import BaseModel
# from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from dotenv import load_dotenv

locals = load_dotenv()


class SubQuery(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    sub_query: str = Field(
        ...,
        description="A very specific query against the database.",
    )


system = f"""You are an expert at converting user questions into database queries. \
You have access to a database of monthly reports. \

Perform query decomposition. Given a user question, break it down into distinct sub questions that \
you need to answer in order to answer the original question.

current month and year : {present_month} {present_year}

If the question have not specify both a year and a month, default to the current month and year: {present_month} {present_year}, \
and produce only one subquery using this default.

- If a specific year is provided, convert it to 4 digits.
- When a time range like "last months" or "last years" is specified, generate a subquery for each relevant period.

If question have only specify a year(e.g: last year), default to the current year:{present_year}, \
and produce only one subquery using this default.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
Settings.llm = llm
llm_with_tools = llm.bind_tools([SubQuery])
parser = PydanticToolsParser(tools=[SubQuery])
query_analyzer = prompt | llm_with_tools | parser

In [3]:
query_str = "tell me about investment objective for Alfalah ghp cash for last 3 month"
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='investment objective for Alfalah GHP Cash for August 2024'),
 SubQuery(sub_query='investment objective for Alfalah GHP Cash for September 2024'),
 SubQuery(sub_query='investment objective for Alfalah GHP Cash for October 2024')]

In [4]:
query_str = "Give me historical Performance % of Alfalah GHP Value Fund"
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='historical Performance % of Alfalah GHP Value Fund for November 2024')]

In [5]:
query_str = "tell me about investment objective for Alfalah ghp cash for last month"
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='investment objective for Alfalah GHP Cash for October 2024')]

In [6]:
query_str = (
    "tell me about the fund performance of Alfalah GHP Stock Fund for last three years."
)
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for the year 2022'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for the year 2023'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for the year 2024')]

In [7]:
query_str = (
    "tell me about the fund performance of Alfalah GHP Stock Fund for last four years."
)
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for the year 2020'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for the year 2021'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for the year 2022'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for the year 2023')]

In [8]:
query_str = (
    "tell me about the fund performance of Alfalah GHP Stock Fund for last two years."
)
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for November 2022'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for November 2023')]

In [9]:
query_str = "What are the best performing funds over the last 4 month?"
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='best performing funds in July 2024'),
 SubQuery(sub_query='best performing funds in August 2024'),
 SubQuery(sub_query='best performing funds in September 2024'),
 SubQuery(sub_query='best performing funds in October 2024')]

In [10]:
query_str = "Which fund is the best performing fund in last year?"
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='best performing fund in 2023')]

In [11]:
query_str = "tell me about the fund performance of Alfalah GHP Stock Fund from may 2024 to september 2024?"
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

[SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for May 2024'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for June 2024'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for July 2024'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for August 2024'),
 SubQuery(sub_query='fund performance of Alfalah GHP Stock Fund for September 2024')]

In [ ]:
query_str = "tell me about the fund performance of Alfalah GHP Stock Fund from may 2024 to september 2024?"
sub_qeuries = query_analyzer.invoke({"question": query_str})
sub_qeuries

In [1]:
arr =  [ 2,4,6,7,8,9]
max = arr[0]
for num in arr:
    if num > max:
        max = num
print(max)

9
